In [11]:
import gc
import sys
import os
import warnings

sys.path.append(os.path.abspath("modules/"))
sys.path.append(os.path.abspath("fe_modules/"))
sys.path.append(os.path.abspath("seq2seq_modules/"))

os.environ['OPENBLAS_NUM_THREADS'] = '1'
warnings.filterwarnings('ignore')

In [12]:
import pandas as pd
import numpy as np
import time
import polars as pl
import scipy
import implicit
import bisect
import sklearn.metrics as m
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve, CalibratedClassifierCV

In [13]:
import importlib

import modules
import fe_modules

importlib.reload(modules)
importlib.reload(fe_modules)

from modules.memory_utils import pandas_reduce_mem_usage, pandas_string_to_cat
from modules.sanity_checks import null_check
from fe_modules.text_manipulation import get_domain
from fe_modules.datetime_features import get_timestamp, \
                                         get_relative_time, \
                                         part_of_day_to_hour, \
                                         add_hour_to_date, \
                                         get_year, \
                                         get_month, \
                                         get_day, \
                                         get_day_of_year, \
                                         get_day_of_week, \
                                         get_holiday_name
from fe_modules.preprocessing import clean_os_type
from fe_modules.missing import map_prices
from fe_modules.geo_features import map_cities, dist_to_large_cities

# Load data

In [4]:
LOCAL_DATA_PATH = './data/'
SPLIT_SEED = 42

In [5]:
df = pandas_reduce_mem_usage( 
        pd.read_parquet(
            f'{LOCAL_DATA_PATH}competition_data_final_pqt/'
    ))
df.head()

Memory usage of dataframe is 29562.33 MB


100%|███████████████████████████████████████████| 12/12 [00:03<00:00,  3.06it/s]

Memory usage after optimization is: 24943.21 MB
Decreased by 15.6%


,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id
0,Краснодарский край,Краснодар,Apple,iPhone 7,ad.adriver.ru,smartphone,iOS,20368.0,2022-06-15,morning,1,45098
1,Краснодарский край,Краснодар,Apple,iPhone 7,apple.com,smartphone,iOS,20368.0,2022-06-19,morning,1,45098
2,Краснодарский край,Краснодар,Apple,iPhone 7,avatars.mds.yandex.net,smartphone,iOS,20368.0,2022-06-12,day,1,45098
3,Краснодарский край,Краснодар,Apple,iPhone 7,googleads.g.doubleclick.net,smartphone,iOS,20368.0,2022-05-16,day,1,45098
4,Краснодарский край,Краснодар,Apple,iPhone 7,googleads.g.doubleclick.net,smartphone,iOS,20368.0,2022-05-30,day,1,45098


# Feature engeneering

## Cleaning

In [7]:
df = clean_os_type(df)
df.head()

TypeError: Series.map() got an unexpected keyword argument 'inplace'

## Feature generation

In [7]:
df = pandas_reduce_mem_usage(map_prices(df))
df.head()

Memory usage of dataframe is 27406.74 MB


100%|███████████████████████████████████████████| 12/12 [00:01<00:00, 10.31it/s]

Memory usage after optimization is: 27406.74 MB
Decreased by 0.0%


,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id
0,Краснодарский край,Краснодар,Apple,iPhone 7,ad.adriver.ru,smartphone,iOS,20368.0,2022-06-15,morning,1,45098
1,Краснодарский край,Краснодар,Apple,iPhone 7,apple.com,smartphone,iOS,20368.0,2022-06-19,morning,1,45098
2,Краснодарский край,Краснодар,Apple,iPhone 7,avatars.mds.yandex.net,smartphone,iOS,20368.0,2022-06-12,day,1,45098
3,Краснодарский край,Краснодар,Apple,iPhone 7,googleads.g.doubleclick.net,smartphone,iOS,20368.0,2022-05-16,day,1,45098
4,Краснодарский край,Краснодар,Apple,iPhone 7,googleads.g.doubleclick.net,smartphone,iOS,20368.0,2022-05-30,day,1,45098


In [8]:
df = get_year(df)
df = get_month(df)
df = get_day(df)
df = pandas_reduce_mem_usage(df)
df.head()

Memory usage of dataframe is 34797.32 MB


100%|███████████████████████████████████████████| 15/15 [00:02<00:00,  6.75it/s]

Memory usage after optimization is: 28638.50 MB
Decreased by 17.7%


,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id,year,month,day
0,Краснодарский край,Краснодар,Apple,iPhone 7,ad.adriver.ru,smartphone,iOS,20368.0,2022-06-15,morning,1,45098,2022,6,15
1,Краснодарский край,Краснодар,Apple,iPhone 7,apple.com,smartphone,iOS,20368.0,2022-06-19,morning,1,45098,2022,6,19
2,Краснодарский край,Краснодар,Apple,iPhone 7,avatars.mds.yandex.net,smartphone,iOS,20368.0,2022-06-12,day,1,45098,2022,6,12
3,Краснодарский край,Краснодар,Apple,iPhone 7,googleads.g.doubleclick.net,smartphone,iOS,20368.0,2022-05-16,day,1,45098,2022,5,16
4,Краснодарский край,Краснодар,Apple,iPhone 7,googleads.g.doubleclick.net,smartphone,iOS,20368.0,2022-05-30,day,1,45098,2022,5,30


In [9]:
df = pandas_string_to_cat(df, 
                            ["region_name", 
                             "year",
                             "cpe_manufacturer_name", 
                             "cpe_type_cd", 
                             "cpe_model_os_type",
                            ]
                           )
df.head()

Memory usage of dataframe is 28638.50 MB
Memory usage of dataframe is 27.97 GB


100%|█████████████████████████████████████████████| 5/5 [00:55<00:00, 11.16s/it]

Memory usage after optimization is: 19708.22 MB
Decreased by 31.2%


,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id,year,month,day
0,21,Краснодар,1,iPhone 7,ad.adriver.ru,2,1,20368.0,2022-06-15,morning,1,45098,1,6,15
1,21,Краснодар,1,iPhone 7,apple.com,2,1,20368.0,2022-06-19,morning,1,45098,1,6,19
2,21,Краснодар,1,iPhone 7,avatars.mds.yandex.net,2,1,20368.0,2022-06-12,day,1,45098,1,6,12
3,21,Краснодар,1,iPhone 7,googleads.g.doubleclick.net,2,1,20368.0,2022-05-16,day,1,45098,1,5,16
4,21,Краснодар,1,iPhone 7,googleads.g.doubleclick.net,2,1,20368.0,2022-05-30,day,1,45098,1,5,30


In [10]:
df.dtypes

region_name                uint8
city_name                 object
cpe_manufacturer_name      uint8
cpe_model_name            object
url_host                  object
cpe_type_cd                uint8
cpe_model_os_type          uint8
price                    float32
date                      object
part_of_day               object
request_cnt                 int8
user_id                    int32
year                       uint8
month                       int8
day                         int8
dtype: object

In [11]:
df.to_parquet("seq2seq_data/stages/stage_1.parquet.gzip",
              compression='gzip')

In [4]:
df = pandas_reduce_mem_usage(pd.read_parquet("seq2seq_data/stages/stage_1.parquet.gzip"))

Memory usage of dataframe is 19708.22 MB


100%|███████████████████████████████████████████| 15/15 [00:02<00:00,  6.57it/s]

Memory usage after optimization is: 19708.22 MB
Decreased by 0.0%


In [5]:
df = get_day_of_year(df)
df = get_day_of_week(df)
df = pandas_reduce_mem_usage(df)
df.head()

Memory usage of dataframe is 24635.27 MB


100%|███████████████████████████████████████████| 17/17 [00:02<00:00,  8.06it/s]

Memory usage after optimization is: 20632.04 MB
Decreased by 16.2%


,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id,year,month,day,day_of_year,day_of_week
0,21,Краснодар,1,iPhone 7,ad.adriver.ru,2,1,20368.0,2022-06-15,morning,1,45098,1,6,15,166,2
1,21,Краснодар,1,iPhone 7,apple.com,2,1,20368.0,2022-06-19,morning,1,45098,1,6,19,170,6
2,21,Краснодар,1,iPhone 7,avatars.mds.yandex.net,2,1,20368.0,2022-06-12,day,1,45098,1,6,12,163,6
3,21,Краснодар,1,iPhone 7,googleads.g.doubleclick.net,2,1,20368.0,2022-05-16,day,1,45098,1,5,16,136,0
4,21,Краснодар,1,iPhone 7,googleads.g.doubleclick.net,2,1,20368.0,2022-05-30,day,1,45098,1,5,30,150,0


In [6]:
df = get_holiday_name(df)
df = pandas_reduce_mem_usage(df)
df.head()

Memory usage of dataframe is 23095.57 MB


100%|███████████████████████████████████████████| 18/18 [00:01<00:00, 10.70it/s]

Memory usage after optimization is: 23095.57 MB
Decreased by 0.0%


,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id,year,month,day,day_of_year,day_of_week,holiday
0,21,Краснодар,1,iPhone 7,ad.adriver.ru,2,1,20368.0,2022-06-15,morning,1,45098,1,6,15,166,2,Не праздник
1,21,Краснодар,1,iPhone 7,apple.com,2,1,20368.0,2022-06-19,morning,1,45098,1,6,19,170,6,Не праздник
2,21,Краснодар,1,iPhone 7,avatars.mds.yandex.net,2,1,20368.0,2022-06-12,day,1,45098,1,6,12,163,6,День России
3,21,Краснодар,1,iPhone 7,googleads.g.doubleclick.net,2,1,20368.0,2022-05-16,day,1,45098,1,5,16,136,0,Не праздник
4,21,Краснодар,1,iPhone 7,googleads.g.doubleclick.net,2,1,20368.0,2022-05-30,day,1,45098,1,5,30,150,0,Не праздник


In [7]:
df = pandas_string_to_cat(df, 
                            [
                             "holiday"
                            ]
                           )

df.head()

Memory usage of dataframe is 23095.57 MB
Memory usage of dataframe is 22.55 GB


100%|█████████████████████████████████████████████| 1/1 [00:11<00:00, 11.93s/it]

Memory usage after optimization is: 20939.98 MB
Decreased by 9.3%


,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id,year,month,day,day_of_year,day_of_week,holiday
0,21,Краснодар,1,iPhone 7,ad.adriver.ru,2,1,20368.0,2022-06-15,morning,1,45098,1,6,15,166,2,5
1,21,Краснодар,1,iPhone 7,apple.com,2,1,20368.0,2022-06-19,morning,1,45098,1,6,19,170,6,5
2,21,Краснодар,1,iPhone 7,avatars.mds.yandex.net,2,1,20368.0,2022-06-12,day,1,45098,1,6,12,163,6,1
3,21,Краснодар,1,iPhone 7,googleads.g.doubleclick.net,2,1,20368.0,2022-05-16,day,1,45098,1,5,16,136,0,5
4,21,Краснодар,1,iPhone 7,googleads.g.doubleclick.net,2,1,20368.0,2022-05-30,day,1,45098,1,5,30,150,0,5


In [9]:
df.to_parquet("seq2seq_data/stages/stage_2.parquet.gzip",
              compression='gzip')

In [4]:
df = pandas_reduce_mem_usage(pd.read_parquet("seq2seq_data/stages/stage_2.parquet.gzip"))
df.head()

Memory usage of dataframe is 20939.98 MB


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:02<00:00,  6.20it/s]

Memory usage after optimization is: 20939.98 MB
Decreased by 0.0%


,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id,year,month,day,day_of_year,day_of_week,holiday
0,21,Краснодар,1,iPhone 7,ad.adriver.ru,2,1,20368.0,2022-06-15,morning,1,45098,1,6,15,166,2,5
1,21,Краснодар,1,iPhone 7,apple.com,2,1,20368.0,2022-06-19,morning,1,45098,1,6,19,170,6,5
2,21,Краснодар,1,iPhone 7,avatars.mds.yandex.net,2,1,20368.0,2022-06-12,day,1,45098,1,6,12,163,6,1
3,21,Краснодар,1,iPhone 7,googleads.g.doubleclick.net,2,1,20368.0,2022-05-16,day,1,45098,1,5,16,136,0,5
4,21,Краснодар,1,iPhone 7,googleads.g.doubleclick.net,2,1,20368.0,2022-05-30,day,1,45098,1,5,30,150,0,5


In [5]:
df = map_cities(df)
df.head()

Memory usage of dataframe is 0.05 MB


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 3202.57it/s]


Memory usage after optimization is: 0.03 MB
Decreased by 47.8%


,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,...,month,day,day_of_year,day_of_week,holiday,capital_marker,timezone,geo_lat,geo_lon,population
0,21,Краснодар,1,iPhone 7,ad.adriver.ru,2,1,20368.0,2022-06-15,morning,...,6,15,166,2,5,2.0,3,45.040161,38.975964,744933
1,21,Краснодар,1,iPhone 7,apple.com,2,1,20368.0,2022-06-19,morning,...,6,19,170,6,5,2.0,3,45.040161,38.975964,744933
2,21,Краснодар,1,iPhone 7,avatars.mds.yandex.net,2,1,20368.0,2022-06-12,day,...,6,12,163,6,1,2.0,3,45.040161,38.975964,744933
3,21,Краснодар,1,iPhone 7,googleads.g.doubleclick.net,2,1,20368.0,2022-05-16,day,...,5,16,136,0,5,2.0,3,45.040161,38.975964,744933
4,21,Краснодар,1,iPhone 7,googleads.g.doubleclick.net,2,1,20368.0,2022-05-30,day,...,5,30,150,0,5,2.0,3,45.040161,38.975964,744933


In [6]:
df.to_parquet("seq2seq_data/stages/stage_3.parquet.gzip",
              compression='gzip')

In [5]:
df = pandas_reduce_mem_usage(pd.read_parquet("seq2seq_data/stages/stage_3.parquet.gzip"))
df.head()

Memory usage of dataframe is 26659.76 MB


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:07<00:00,  3.03it/s]

Memory usage after optimization is: 26659.76 MB
Decreased by 0.0%


,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,...,month,day,day_of_year,day_of_week,holiday,capital_marker,timezone,geo_lat,geo_lon,population
0,21,Краснодар,1,iPhone 7,ad.adriver.ru,2,1,20368.0,2022-06-15,morning,...,6,15,166,2,5,2.0,3,45.040161,38.975964,744933
1,21,Краснодар,1,iPhone 7,apple.com,2,1,20368.0,2022-06-19,morning,...,6,19,170,6,5,2.0,3,45.040161,38.975964,744933
2,21,Краснодар,1,iPhone 7,avatars.mds.yandex.net,2,1,20368.0,2022-06-12,day,...,6,12,163,6,1,2.0,3,45.040161,38.975964,744933
3,21,Краснодар,1,iPhone 7,googleads.g.doubleclick.net,2,1,20368.0,2022-05-16,day,...,5,16,136,0,5,2.0,3,45.040161,38.975964,744933
4,21,Краснодар,1,iPhone 7,googleads.g.doubleclick.net,2,1,20368.0,2022-05-30,day,...,5,30,150,0,5,2.0,3,45.040161,38.975964,744933


In [6]:
df = pandas_reduce_mem_usage(dist_to_large_cities(df))
df.head()

  0%|          | 0/5 [00:00<?, ?it/s]

Memory usage of dataframe is 0.06 MB


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 4552.22it/s]

Memory usage after optimization is: 0.04 MB
Decreased by 31.2%


Memory usage of dataframe is 35441.80 MB


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:12<00:00,  2.31it/s]

Memory usage after optimization is: 35441.80 MB
Decreased by 0.0%


,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,...,geo_lat_x,geo_lon_x,population,geo_lat_y,geo_lon_y,dist_to_Moscow,dist_to_SaintP,dist_to_Novosibirsk,dist_to_Ekaterinburg,dist_to_Vladivostok
0,21,Краснодар,1,iPhone 7,ad.adriver.ru,2,1,20368.0,2022-06-15,morning,...,45.040161,38.975964,744933,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391
1,21,Краснодар,1,iPhone 7,apple.com,2,1,20368.0,2022-06-19,morning,...,45.040161,38.975964,744933,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391
2,21,Краснодар,1,iPhone 7,avatars.mds.yandex.net,2,1,20368.0,2022-06-12,day,...,45.040161,38.975964,744933,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391
3,21,Краснодар,1,iPhone 7,googleads.g.doubleclick.net,2,1,20368.0,2022-05-16,day,...,45.040161,38.975964,744933,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391
4,21,Краснодар,1,iPhone 7,googleads.g.doubleclick.net,2,1,20368.0,2022-05-30,day,...,45.040161,38.975964,744933,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391


In [8]:
df.to_parquet("seq2seq_data/stages/stage_4.parquet.gzip",
              compression='gzip')

In [4]:
df = pandas_reduce_mem_usage(pd.read_parquet("seq2seq_data/stages/stage_4.parquet.gzip"))
df.head()

Memory usage of dataframe is 31678.07 MB


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:19<00:00,  1.57it/s]

Memory usage after optimization is: 31678.07 MB
Decreased by 0.0%


,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,...,geo_lat_x,geo_lon_x,population,geo_lat_y,geo_lon_y,dist_to_Moscow,dist_to_SaintP,dist_to_Novosibirsk,dist_to_Ekaterinburg,dist_to_Vladivostok
0,21,409,1,589,ad.adriver.ru,2,1,20368.0,2022-06-15,morning,...,45.040161,38.975964,744933,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391
1,21,409,1,589,apple.com,2,1,20368.0,2022-06-19,morning,...,45.040161,38.975964,744933,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391
2,21,409,1,589,avatars.mds.yandex.net,2,1,20368.0,2022-06-12,day,...,45.040161,38.975964,744933,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391
3,21,409,1,589,googleads.g.doubleclick.net,2,1,20368.0,2022-05-16,day,...,45.040161,38.975964,744933,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391
4,21,409,1,589,googleads.g.doubleclick.net,2,1,20368.0,2022-05-30,day,...,45.040161,38.975964,744933,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391


In [5]:
df = pandas_string_to_cat(df, 
                            [
                            "cpe_model_name",
                            "city_name", 
                            ]
                           )
df.head()

Memory usage of dataframe is 31678.07 MB
Memory usage of dataframe is 30.94 GB


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:04<00:00,  2.50s/it]

Memory usage after optimization is: 31678.07 MB
Decreased by 0.0%


,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,...,geo_lat_x,geo_lon_x,population,geo_lat_y,geo_lon_y,dist_to_Moscow,dist_to_SaintP,dist_to_Novosibirsk,dist_to_Ekaterinburg,dist_to_Vladivostok
0,21,409,1,589,ad.adriver.ru,2,1,20368.0,2022-06-15,morning,...,45.040161,38.975964,744933,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391
1,21,409,1,589,apple.com,2,1,20368.0,2022-06-19,morning,...,45.040161,38.975964,744933,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391
2,21,409,1,589,avatars.mds.yandex.net,2,1,20368.0,2022-06-12,day,...,45.040161,38.975964,744933,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391
3,21,409,1,589,googleads.g.doubleclick.net,2,1,20368.0,2022-05-16,day,...,45.040161,38.975964,744933,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391
4,21,409,1,589,googleads.g.doubleclick.net,2,1,20368.0,2022-05-30,day,...,45.040161,38.975964,744933,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391


In [6]:
df = get_domain(df)
df.head()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 199683/199683 [00:00<00:00, 1744533.21it/s]


,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,...,geo_lon_x,population,geo_lat_y,geo_lon_y,dist_to_Moscow,dist_to_SaintP,dist_to_Novosibirsk,dist_to_Ekaterinburg,dist_to_Vladivostok,domain
0,21,409,1,589,ad.adriver.ru,2,1,20368.0,2022-06-15,morning,...,38.975964,744933,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,ru
1,21,409,1,589,apple.com,2,1,20368.0,2022-06-19,morning,...,38.975964,744933,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,com
2,21,409,1,589,avatars.mds.yandex.net,2,1,20368.0,2022-06-12,day,...,38.975964,744933,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,net
3,21,409,1,589,googleads.g.doubleclick.net,2,1,20368.0,2022-05-16,day,...,38.975964,744933,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,net
4,21,409,1,589,googleads.g.doubleclick.net,2,1,20368.0,2022-05-30,day,...,38.975964,744933,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,net


In [7]:
df = pandas_string_to_cat(df, 
                            [
                            "domain",
                            "url_host"
                            ]
                           )
df.head()

Memory usage of dataframe is 34187.23 MB
Memory usage of dataframe is 33.39 GB


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:24<00:00, 12.18s/it]

Memory usage after optimization is: 31050.78 MB
Decreased by 9.2%


,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,...,geo_lon_x,population,geo_lat_y,geo_lon_y,dist_to_Moscow,dist_to_SaintP,dist_to_Novosibirsk,dist_to_Ekaterinburg,dist_to_Vladivostok,domain
0,21,409,1,589,5788,2,1,20368.0,2022-06-15,morning,...,38.975964,744933,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,761
1,21,409,1,589,12900,2,1,20368.0,2022-06-19,morning,...,38.975964,744933,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,549
2,21,409,1,589,17626,2,1,20368.0,2022-06-12,day,...,38.975964,744933,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,712
3,21,409,1,589,59366,2,1,20368.0,2022-05-16,day,...,38.975964,744933,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,712
4,21,409,1,589,59366,2,1,20368.0,2022-05-30,day,...,38.975964,744933,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,712


In [8]:
df.to_parquet("seq2seq_data/stages/stage_5.parquet.gzip",
              compression='gzip')

In [4]:
df = pandas_reduce_mem_usage(pd.read_parquet("seq2seq_data/stages/stage_5.parquet.gzip"))

Memory usage of dataframe is 31050.78 MB


  0%|          | 0/31 [00:00<?, ?it/s]

Memory usage after optimization is: 31050.78 MB
Decreased by 0.0%


In [5]:
df = part_of_day_to_hour(df)
df.head()

,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,...,population,geo_lat_y,geo_lon_y,dist_to_Moscow,dist_to_SaintP,dist_to_Novosibirsk,dist_to_Ekaterinburg,dist_to_Vladivostok,domain,hour
0,21,409,1,589,5788,2,1,20368.0,2022-06-15,morning,...,744933,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,761,0 days 09:00:00
1,21,409,1,589,12900,2,1,20368.0,2022-06-19,morning,...,744933,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,549,0 days 09:00:00
2,21,409,1,589,17626,2,1,20368.0,2022-06-12,day,...,744933,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,712,0 days 15:00:00
3,21,409,1,589,59366,2,1,20368.0,2022-05-16,day,...,744933,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,712,0 days 15:00:00
4,21,409,1,589,59366,2,1,20368.0,2022-05-30,day,...,744933,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,712,0 days 15:00:00


In [6]:
df = add_hour_to_date(df)
df.head()

,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,...,geo_lat_y,geo_lon_y,dist_to_Moscow,dist_to_SaintP,dist_to_Novosibirsk,dist_to_Ekaterinburg,dist_to_Vladivostok,domain,hour,datetime
0,21,409,1,589,5788,2,1,20368.0,2022-06-15,morning,...,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,761,0 days 09:00:00,2022-06-15 09:00:00
1,21,409,1,589,12900,2,1,20368.0,2022-06-19,morning,...,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,549,0 days 09:00:00,2022-06-19 09:00:00
2,21,409,1,589,17626,2,1,20368.0,2022-06-12,day,...,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,712,0 days 15:00:00,2022-06-12 15:00:00
3,21,409,1,589,59366,2,1,20368.0,2022-05-16,day,...,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,712,0 days 15:00:00,2022-05-16 15:00:00
4,21,409,1,589,59366,2,1,20368.0,2022-05-30,day,...,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,712,0 days 15:00:00,2022-05-30 15:00:00


In [7]:
def my_reset(*varnames):
    """
    varnames are what you want to keep
    """
    globals_ = globals()
    to_save = {v: globals_[v] for v in varnames}
    to_save['my_reset'] = my_reset  # lets keep this function by default
    del globals_
    get_ipython().magic("reset")
    globals().update(to_save)

In [8]:
df = df.drop(labels=["date", "hour"], axis=1)
gc.collect()
df.head()

,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,part_of_day,request_cnt,...,population,geo_lat_y,geo_lon_y,dist_to_Moscow,dist_to_SaintP,dist_to_Novosibirsk,dist_to_Ekaterinburg,dist_to_Vladivostok,domain,datetime
0,21,409,1,589,5788,2,1,20368.0,morning,1,...,744933,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,761,2022-06-15 09:00:00
1,21,409,1,589,12900,2,1,20368.0,morning,1,...,744933,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,549,2022-06-19 09:00:00
2,21,409,1,589,17626,2,1,20368.0,day,1,...,744933,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,712,2022-06-12 15:00:00
3,21,409,1,589,59366,2,1,20368.0,day,1,...,744933,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,712,2022-05-16 15:00:00
4,21,409,1,589,59366,2,1,20368.0,day,1,...,744933,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,712,2022-05-30 15:00:00


In [9]:
df = pandas_reduce_mem_usage(get_timestamp(df), columns=["timestamp"])
df.head()

Memory usage of dataframe is 33559.94 MB


  0%|          | 0/1 [00:00<?, ?it/s]

Memory usage after optimization is: 32305.36 MB
Decreased by 3.7%


,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,part_of_day,request_cnt,...,geo_lat_y,geo_lon_y,dist_to_Moscow,dist_to_SaintP,dist_to_Novosibirsk,dist_to_Ekaterinburg,dist_to_Vladivostok,domain,datetime,timestamp
0,21,409,1,589,5788,2,1,20368.0,morning,1,...,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,761,2022-06-15 09:00:00,165528352.0
1,21,409,1,589,12900,2,1,20368.0,morning,1,...,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,549,2022-06-19 09:00:00,165562912.0
2,21,409,1,589,17626,2,1,20368.0,day,1,...,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,712,2022-06-12 15:00:00,165504608.0
3,21,409,1,589,59366,2,1,20368.0,day,1,...,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,712,2022-05-16 15:00:00,165271328.0
4,21,409,1,589,59366,2,1,20368.0,day,1,...,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,712,2022-05-30 15:00:00,165392288.0


In [10]:
my_reset("df")

Once deleted, variables cannot be recovered. Proceed (y/[n])? н
Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [14]:
df = pandas_reduce_mem_usage(get_relative_time(df).drop(labels=["datetime"], axis=1), columns=["relative_timestamp"])
df.head()

Memory usage of dataframe is 4.75 MB


  0%|          | 0/1 [00:00<?, ?it/s]

Memory usage after optimization is: 4.75 MB
Decreased by 0.0%
Memory usage of dataframe is 33559.94 MB


  0%|          | 0/1 [00:00<?, ?it/s]

Memory usage after optimization is: 33559.94 MB
Decreased by 0.0%
Memory usage of dataframe is 31050.78 MB


  0%|          | 0/1 [00:00<?, ?it/s]

Memory usage after optimization is: 31050.78 MB
Decreased by 0.0%


,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,part_of_day,request_cnt,...,geo_lat_y,geo_lon_y,dist_to_Moscow,dist_to_SaintP,dist_to_Novosibirsk,dist_to_Ekaterinburg,dist_to_Vladivostok,domain,timestamp,relative_timestamp
0,21,409,1,589,5788,2,1,20368.0,morning,1,...,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,761,165528352.0,388.799988
1,21,409,1,589,12900,2,1,20368.0,morning,1,...,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,549,165562912.0,43.200001
2,21,409,1,589,17626,2,1,20368.0,day,1,...,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,712,165504608.0,626.239990
3,21,409,1,589,59366,2,1,20368.0,day,1,...,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,712,165271328.0,2959.040039
4,21,409,1,589,59366,2,1,20368.0,day,1,...,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,712,165392288.0,1749.439941


In [15]:
df = pandas_string_to_cat(df, 
                            [
                            "part_of_day"
                            ]
                           )
df.head()

Memory usage of dataframe is 31050.78 MB
Memory usage of dataframe is 30.32 GB


  0%|          | 0/1 [00:00<?, ?it/s]

Memory usage after optimization is: 28855.27 MB
Decreased by 7.1%


,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,part_of_day,request_cnt,...,geo_lat_y,geo_lon_y,dist_to_Moscow,dist_to_SaintP,dist_to_Novosibirsk,dist_to_Ekaterinburg,dist_to_Vladivostok,domain,timestamp,relative_timestamp
0,21,409,1,589,5788,2,1,20368.0,2,1,...,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,761,165528352.0,388.799988
1,21,409,1,589,12900,2,1,20368.0,2,1,...,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,549,165562912.0,43.200001
2,21,409,1,589,17626,2,1,20368.0,0,1,...,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,712,165504608.0,626.239990
3,21,409,1,589,59366,2,1,20368.0,0,1,...,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,712,165271328.0,2959.040039
4,21,409,1,589,59366,2,1,20368.0,0,1,...,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,712,165392288.0,1749.439941


In [16]:
df.to_parquet("seq2seq_data/stages/stage_6.parquet.gzip",
              compression='gzip')